In [ ]:
%pip install numpy

In [ ]:
import numpy as np
import random

In [ ]:

#Base class, any child classes should implement evolve and have a value
class Asset:
    def __init__(self, initialValue):
        self.value = initialValue
    def evolve(self):
        self.value = self.value

class Bond(Asset):
    #TODO: add conditions to ensure rate is valid
    def __init__(self, initialValue, rate):
        self.rate = rate
        super().__init__(initialValue)
    def evolve(self):
        self.value = self.value * self.rate

class BinomialStock(Asset):
    def __init__(self, initialValue, u, d, p):
        self.u = u
        self.d = d
        self.p = p
        super().__init__(initialValue)
    def evolve(self):
        probDraw = random.random()
        if probDraw < self.p:
            self.value += self.u
        else:
            self.value += self.d
class Market:
    def __init__(self, assets):
        self.assets = assets
        self.time = 0
        self.history = []
        self.history.append(self.returnAssetValues())

    def returnAssetValues(self):
        values = []
        for asset in self.assets:
            values.append(asset.value)
        return values

    def evolve(self):
        for asset in self.assets:
            asset.evolve()
        self.history.append(self.returnAssetValues())
        self.time += 1

class Portfolio:

    tradingStrategy = None
    market = None

    currentAllocation = []

    def __init__(self, tradingStrategy, market, initalAllocation):
        self.tradingStrategy = tradingStrategy
        self.market = market
        self.currentAllocation = initalAllocation
    def stepTime(self):
        self.evolve()
        self.rebalance()

    def rebalance(self):
        newAllocation = self.tradingStrategy(self.market.history)
        currentAssetValues = self.market.history[len(self.market.history)-1]

        if (np.dot(self.currentAllocation, currentAssetValues) != np.dot(newAllocation, currentAssetValues)):
            raise ValueError("Value after rebalancing portofolio changed")
        else:
            self.currentAllocation = newAllocation

    def evolve(self):
        self.market.evolve()

    def returnCurrentValue(self):
        currentAssetValues = self.market.history[len(self.market.history)-1]

        return np.dot(currentAssetValues, self.currentAllocation)
        

    

In [ ]:
#Define the assets
assets = [Bond(5, 1.01)]
for i in range(4):
    assets.append(BinomialStock(4, 2,-1, 0.65))

market = Market(assets)

startingAllocation = np.ones(5)

def exampleTradingStrategy(history):
    #Return a constant uniform allocation 
    return np.ones(5)

examplePortfolio = Portfolio(exampleTradingStrategy, market, startingAllocation)
#Initial value of the portfolio
print(examplePortfolio.returnCurrentValue())
for i in range(50):
    examplePortfolio.stepTime()
#Current market value of the assets
print(examplePortfolio.market.returnAssetValues())
#Current allocation of assets within the portfolio
print(examplePortfolio.currentAllocation)
#Current total value of the portfolio
print(examplePortfolio.returnCurrentValue())

The core model